Run in Colab: https://colab.research.google.com/drive/1K1aK-q2XbFLHzU08xVBtnb36HleR5mG_?usp=sharing

# Convolutional Neural Networks
 This code defines a Convolutional Neural Network (CNN) model using PyTorch. The model is used for image classification tasks.

# Model architecture
## Convolutional Layers
 The CNN consists of two convolutional layers with ReLU activation functions and max pooling layers. The first convolutional layer takes a 3-channel input image and outputs 6 feature maps, using a 4x4 kernel. The second convolutional layer takes the output of the first convolutional layer as input and outputs 9 feature maps, using a 5x5 kernel.
## Linear Layers
 The output from the last convolutional layer is flattened into a 1D tensor and passed through two fully connected (linear) layers with ReLU activation functions. The first linear layer has 80 neurons and the second has 10 neurons (the number of classes in the classification task).

# Model Definition

In [ ]:
import torch.nn as nn


class ConvModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 6, 4),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(6, 9, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.linear = nn.Sequential(
            nn.Linear(5 * 5 * 9, 80),
            nn.ReLU(),
            nn.Linear(80, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 5 * 5 * 9)
        x = self.linear(x)
        return x


In the **forward** method, the input tensor is first passed through the convolutional layers and then flattened into a 1D tensor. The flattened tensor is then passed through the linear layers to produce the final output.

# CNN Training

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from model import ConvModel
import torchvision.transforms as tf


def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    learning_rate = 3e-4
    batch_size = 4
    num_epochs = 20

    model = ConvModel().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    transforms = tf.Compose([
        tf.ToTensor(),
        tf.Normalize((0.5,), (0.5,), (0.5,))
    ])

    train_dataset = datasets.CIFAR10("dataset/", train=True, transform=transforms, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = datasets.CIFAR10("dataset/", train=False, transform=transforms, download=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    step = 0
    total_steps = len(train_loader) * num_epochs

    for epoch in range(num_epochs):
        for idx, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (idx + 1) % 2000 == 0:
                print(f'Epoch: {epoch + 1}/{num_epochs} | Step: {step + 1};{total_steps} | Loss: {loss}')

            step += 1

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for _ in range(10)]
        n_class_samples = [0 for _ in range(10)]

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            output = model(images)
            _, predicted = torch.max(output, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

            for i in range(batch_size):
                label = labels[i]
                pred = predicted[i]

                if label == pred:
                    n_class_correct[label.item()] += 1

                n_class_samples[label.item()] += 1

        print(f'Training data accuracy is:  {n_correct / n_samples}')

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for _ in range(10)]
        n_class_samples = [0 for _ in range(10)]

        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            output = model(images)
            _, predicted = torch.max(output, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

            for i in range(batch_size):
                label = labels[i]
                pred = predicted[i]

                if label == pred:
                    n_class_correct[label.item()] += 1

                n_class_samples[label.item()] += 1

        print(f'Test data accuracy is:  {n_correct / n_samples}')


if __name__ == "__main__":
    train()


Files already downloaded and verified
Files already downloaded and verified
Epoch: 1/20 | Step: 2000;250000 | Loss: 1.868149995803833
Epoch: 1/20 | Step: 4000;250000 | Loss: 1.808663249015808
Epoch: 1/20 | Step: 6000;250000 | Loss: 2.0486741065979004
Epoch: 1/20 | Step: 8000;250000 | Loss: 2.148817539215088
Epoch: 1/20 | Step: 10000;250000 | Loss: 1.7136116027832031
Epoch: 1/20 | Step: 12000;250000 | Loss: 1.2580480575561523
Epoch: 2/20 | Step: 14500;250000 | Loss: 0.9803611636161804
Epoch: 2/20 | Step: 16500;250000 | Loss: 0.6896300911903381
Epoch: 2/20 | Step: 18500;250000 | Loss: 0.8863891959190369
Epoch: 2/20 | Step: 20500;250000 | Loss: 1.2686976194381714
Epoch: 2/20 | Step: 22500;250000 | Loss: 0.6618356704711914
Epoch: 2/20 | Step: 24500;250000 | Loss: 1.5075292587280273
Epoch: 3/20 | Step: 27000;250000 | Loss: 1.0405328273773193
Epoch: 3/20 | Step: 29000;250000 | Loss: 1.1788005828857422
Epoch: 3/20 | Step: 31000;250000 | Loss: 0.28276947140693665
Epoch: 3/20 | Step: 33000;2500